---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

Load text data

In [1]:
import pandas as pd

doc = []
with open('../materials/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
len(df)

500

Validate Regex to extract Dates

In [3]:
#from 1 to 193
df.str.findall('\d{1,2}[-/ ](?:\d{1,2}|Jan[A-Za-z]*|Feb[A-Za-z]*|Mar[A-Za-z]*|Apr[A-Za-z]*|May[A-Za-z]*|' +
               'Jun[A-Za-z]*|Jul[A-Za-z]*|Aug[A-Za-z]*|Sep[A-Za-z]*|Oct[A-Za-z]*|Nov[A-Za-z]*|Dec[A-Za-z]*)' + 
               '[-/ ](?:\d{2}|\d{4})(?!\d)').head(200).tail(50)

150     [12 March 1980]
151      [22 June 1990]
152       [28 Sep 2015]
153       [13 Jan 1972]
154       [06 Mar 1974]
155       [10 Oct 1974]
156       [26 May 1974]
157       [10 Feb 1990]
158       [23 Aug 2000]
159       [26 May 2001]
160       [21 Oct 2007]
161       [19 Oct 2016]
162       [05 Mar 1974]
163       [29 Jan 1994]
164       [21 Oct 1978]
165    [18 August 1975]
166       [11 Nov 1996]
167       [01 Oct 1979]
168       [13 Oct 1986]
169       [21 Oct 1995]
170       [24 Jan 2011]
171       [04 Oct 1972]
172       [23 Aug 1993]
173       [18 Oct 2006]
174       [04 Dec 1988]
175       [21 Oct 1983]
176       [26 May 2010]
177       [18 Jan 1990]
178       [15 Jun 1985]
179       [10 Dec 1982]
180       [09 Dec 1988]
181    [18 August 1995]
182      [13 June 1974]
183       [26 May 2008]
184       [11 Nov 2002]
185       [17 Aug 1985]
186       [13 Oct 2016]
187       [14 Jan 2008]
188     [12 March 2004]
189       [21 Oct 1977]
190       [10 Aug 2000]
191       [30 No

In [4]:
#from 194 to 342
df.str.findall('(?:Jan[A-Za-z.]*|Feb[A-Za-z.]*|Mar[A-Za-z.]*|Apr[A-Za-z.]*|May[A-Za-z.]*|Jun[A-Za-z.]*|' +
               'Jul[A-Za-z.]*|Aug[A-Za-z.]*|Sep[A-Za-z.]*|Oct[A-Za-z.]*|Nov[A-Za-z.]*|Dec[A-Za-z.]*)' + 
               '[ ]?(?:\d{0}|\d{2})[,.]? (?:\d{2}|\d{4})(?!\d)').head(350).tail(50)

300      [January 1994]
301          [Dec 1992]
302     [November 2004]
303      [January 1977]
304          [Mar 2002]
305          [Feb 2000]
306         [May, 2004]
307         [July 2006]
308          [Feb 1994]
309        [April 1977]
310          [Oct 1992]
311    [February, 1995]
312     [February 1989]
313    [Decemeber 1978]
314      [January 2007]
315          [Jun 1976]
316          [May 2011]
317         [Mar, 1975]
318          [Jan 1978]
319         [July 1975]
320     [November 2012]
321        [June, 1999]
322      [October 1991]
323        [March 1973]
324      [October 1996]
325          [Jun 2007]
326      [October 1995]
327        [April 1999]
328         [May, 2001]
329       [March, 2000]
330        [April 1988]
331     [December 1993]
332         [June 1974]
333     [November 1997]
334         [July 1986]
335     [February 1973]
336        [March 1978]
337          [Dec 2007]
338         [Apr, 1998]
339       [March, 2005]
340          [May 1980]
341          [No

In [5]:
#from 343 to 500
df.str.findall('\d{0,2}/?\d{4}').head(500).tail(50)

450     [1/1994]
451    [12/2004]
452     [3/2003]
453     [7/1991]
454     [7/1982]
455       [1984]
456       [2000]
457       [2001]
458       [1982]
459       [1998]
460       [2012]
461       [1991]
462       [1988]
463       [2014]
464       [2016]
465       [1976]
466       [1981]
467       [2011]
468       [1997]
469       [2003]
470       [1983]
471       [1999]
472       [2010]
473       [1975]
474       [1972]
475       [2015]
476       [1989]
477       [1994]
478       [1993]
479       [1996]
480       [2013]
481       [1974]
482       [1990]
483       [1995]
484       [2004]
485       [1987]
486       [1973]
487       [1992]
488       [1977]
489       [1985]
490       [2007]
491       [2009]
492       [1986]
493       [1978]
494       [2002]
495       [1979]
496       [2006]
497       [2008]
498       [2005]
499       [1980]
dtype: object

Apply regex, clean dates and sort

In [6]:
import dateutil.parser as parser
import datetime

def date_sorter():
    
    # Define regex to extract dates in different format
    
    regex_1 =  '\d{1,2}[-/ ](?:\d{1,2}|Jan[A-Za-z]*|Feb[A-Za-z]*|Mar[A-Za-z]*|Apr[A-Za-z]*|May[A-Za-z]*|'     + \
               'Jun[A-Za-z]*|Jul[A-Za-z]*|Aug[A-Za-z]*|Sep[A-Za-z]*|Oct[A-Za-z]*|Nov[A-Za-z]*|Dec[A-Za-z]*)' + \
               '[-/ ](?:\d{2}|\d{4})(?!\d)'
            
    regex_2 =  '(?:Jan[A-Za-z.]*|Feb[A-Za-z.]*|Mar[A-Za-z.]*|Apr[A-Za-z.]*|May[A-Za-z.]*|Jun[A-Za-z.]*|' + \
               'Jul[A-Za-z.]*|Aug[A-Za-z.]*|Sep[A-Za-z.]*|Oct[A-Za-z.]*|Nov[A-Za-z.]*|Dec[A-Za-z.]*)'   + \
               '[ ]?(?:\d{0}|\d{2})[,.]? (?:\d{2}|\d{4})(?!\d)'
            
    regex_3 = '\d{0,2}/?\d{4}'
    
    # Apply regex to each of the three buckets and concat
    dates = df.iloc[:194].str.findall(regex_1)
    dates = pd.concat([dates, df.iloc[194:343].str.findall(regex_2)])
    dates = pd.concat([dates, df.iloc[343:].str.findall(regex_3)])
    
    # Convert each elemnt from list to string
    dates = dates.apply(lambda x: x[0])
    
    # Clean typos
    dates = dates.str.replace('Janaury', 'January')
    dates = dates.str.replace('Decemeber', 'December')
    
    # Convert from string to datetime 
    dates = dates.apply(lambda x: parser.parse(x, default = datetime.datetime(2020, 1, 1, 0, 0)))
    
    # Sort dates and return indexes
    sorted_indexes = dates.sort_values(ascending = True).index.values
    
    return pd.Series(sorted_indexes)

In [7]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64